# Prosumer Load Shifting ABM model

## Model structure
#### Data preparation
- create_agent_data

#### Model setup

- create_agents
- connect_agents / create_network

#### Agent logic

- shift_load
- calculate_behaviour

#### Simulation

- simulation 
- save_data

In [5]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import itertools
import random
import copy
import pandas as pd
import networkx as nx
from matplotlib import animation
#import seaborn as sns
plt.style.use("seaborn-dark-palette")

## Izdelava agentovih podatkov

In [6]:
def create_agent_data(number_of_agents):
    #load schedhule and load data
    data = pd.read_excel(r'/data/workspace_files/basic_load_profile.xlsx')
    data = data.set_index('time')
    #store data in schedule dictionary key=hour, pair= load [kW]
    schedule = {}

    for index, row in data.iterrows():
        schedule[index] = row['load_1']
    print(schedule)

    # create groups
    groups = ['eco', 'normal', 'trad']
    # create dataframe to store agent info
    data_frame = pd.DataFrame(columns= ['id'])

    # for each agent add following attributes
    for agent in range(number_of_agents):
        group = groups[random.randint(0,len(groups)-1)] #choose random group from group list
        group_weight = 0 # assign initial group weight
        # assign weights for each group
        if group == 'eco':
            group_weight = 1
            flexibility_coeficient = 0.8
            conv_weight = 0.5
            price_weight = 0.5
        elif group == 'normal':
            group_weight = 0.5
            flexibility_coeficient = 0.5
            conv_weight = 1
            price_weight = 0.5
        elif group == 'trad':
            group_weight = 0.1
            flexibility_coeficient = 0.3
            conv_weight = 1
            price_weight = 1



        data = {'id': agent,
                'schedule': schedule,
                'group_weight': group_weight,
                'flexibility_coeficient': flexibility_coeficient,
                'susceptibility': 0.1,
                'group': group,
                #'group_weight': group_weight,
                'shifted_hours': 0,
                'conv_weight': conv_weight,
                'price_weight': price_weight
                }

        data_frame = data_frame.append(data, ignore_index=True)
    #print(data_frame.index.tolist())
    return data_frame.set_index('id')# Return dataframve with index ID 

In [10]:
def create_agent_data2(num_eco,num_trad,num_normal):
    #load schedhule and load data
    data = pd.read_excel(r'./data/workspace_files/basic_load_profile.xlsx')
    data = data.set_index('time')
    #store data in schedule dictionary key=hour, pair= load [kW]
    schedule = {}
    susceptibility = 0.1

    for index, row in data.iterrows():
        schedule[index] = row['load_1']
    print(schedule)

    # create groups
    groups = ['eco', 'normal', 'trad']
    # create dataframe to store agent info
    data_frame = pd.DataFrame(columns= ['id'])

    # for each agent add following attributes
    for agent in range(num_eco):
        group = groups[0]
        group_weight = 0
        group_weight = 1
        flexibility_coeficient = 0.8
        conv_weight = 0.5
        price_weight = 0.5

        data = {
            'schedule': schedule,
            'group_weight': group_weight,
            'flexibility_coeficient': flexibility_coeficient,
            'susceptibility': susceptibility,
            'group': group,
            #'group_weight': group_weight,
            'shifted_hours': 0,
            'conv_weight': conv_weight,
            'price_weight': price_weight
        }
        data_frame = data_frame.append(data, ignore_index=True)

    for agent in range(num_normal):
        group = groups[1]
        group_weight = 0.5
        flexibility_coeficient = 0.5
        conv_weight = 1
        price_weight = 0.5

        data = {
            'schedule': schedule,
            'group_weight': group_weight,
            'flexibility_coeficient': flexibility_coeficient,
            'susceptibility': susceptibility,
            'group': group,
            #'group_weight': group_weight,
            'shifted_hours': 0,
            'conv_weight': conv_weight,
            'price_weight': price_weight
            }
        data_frame = data_frame.append(data, ignore_index=True)

    for agent in range(num_trad):
        group = groups[2]
        group_weight = 0.1
        flexibility_coeficient = 0.3
        conv_weight = 1
        price_weight = 1

        data = {'schedule': schedule,
                'group_weight': group_weight,
                'flexibility_coeficient': flexibility_coeficient,
                'susceptibility': susceptibility,
                'group': group,
                #'group_weight': group_weight,
                'shifted_hours': 0,
                'conv_weight': conv_weight,
                'price_weight': price_weight
                }
        

        data_frame = data_frame.append(data, ignore_index=True)

    data_frame.insert(0,'New_id', range(0,len(data_frame)))
    #print(data_frame.index.tolist())
    return data_frame.set_index('New_id')# Return dataframve with index ID 

In [11]:
# prepare data with function create_agent_data and save it in variable 
#new_agent_data = create_agent_data(12)
new_agent_data = create_agent_data2(4,4,4)
new_agent_data.to_csv('random_agentdata.csv')
new_agent_data

ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

## Funkcija za izdelavo agentov

In [9]:
#create empty graph for further use
consumer_network = nx.Graph()

def create_agents(agent_data, network):
    for index, row in agent_data.iterrows(): # for each row in agent data add new node to graph and add data
        # for each agent, add new node
        network.add_node(index)
        #network.nodes[index]['schedule'] = row['schedule']
        network.nodes[index]['schedule'] = {1: 'empty', 2: 'empty', 3: 'empty', 4: 'empty', 5: 'empty', 6: 'empty', 7: 'empty', 8: 'empty', 9: 'empty', 10: 'empty', 11: 'empty', 12: 'empty', 13: 'empty', 14: 'empty', 15: 'empty', 16: 'empty', 17: 'empty', 18: 'flexible', 19: 'empty', 20: 'empty', 21: 'empty', 22: 'empty', 23: 'empty', 24: 'empty'}
        network.nodes[index]['flexibility_coeficient'] = row['flexibility_coeficient']
        network.nodes[index]['susceptibility'] = row['susceptibility']
        network.nodes[index]['group'] = row['group']
        network.nodes[index]['group_weight'] = row['group_weight']
        network.nodes[index]['shifted_hours'] = row['shifted_hours']
        network.nodes[index]['conv_weight'] = row['conv_weight']
        network.nodes[index]['price_weight'] = row['price_weight']

    #set network position to circle
    pos = nx.random_layout(network)
    #draw graph
    plt.figure(figsize = (9, 6)) 
    print(network.nodes.data())
    nx.draw_networkx(network,pos)
    return network

In [ ]:
test = create_agents(new_agent_data, consumer_network)
test.nodes.data()

In [ ]:
test.nodes[1]

# Poveži agente v omrežje

In [ ]:
def connect_agents(network):
    # save agents to list 
    nodesList = list(network.__iter__())
    # loop trough agents
    for node in network.__iter__():
        # add edge to agent from a list of all agents
        network.add_edge(node, nodesList[random.randint(0,len(nodesList)-1)])
        network.add_edge(node, nodesList[random.randint(0,len(nodesList)-1)])
        # add list of all neighbours to agent atributes
        network.nodes[node]['neighbours'] = list(network.neighbors(node))
    # draw network graph
    pos = nx.circular_layout(network)
    plt.figure(figsize = (9, 9))
    #nx.draw_networkx_edges(network, pos)
    #nx.draw_networkx_nodes(network, pos, node_size=130, node_color='white', alpha=0.9, linewidths=1, edgecolors='gray')
    #plt.axis("off")
    #plt.show()
    nx.draw_networkx(network, pos)  
    return network

In [ ]:
# connect agents on test network and check if it works
test = connect_agents(test)
print(test.edges.data)

## Odločitvena logika agentov 

In [ ]:
test_schedule = {1: 'empty', 2: 'empty', 3: 'empty', 4: 'empty', 5: 'empty', 6: 'empty', 7: 'empty', 8: 'empty', 9: 'empty', 10: 'empty', 11: 'empty', 12: 'empty', 13: 'empty', 14: 'empty', 15: 'empty', 16: 'empty', 17: 'empty', 18: 'flexible', 19: 'empty', 20: 'empty', 21: 'empty', 22: 'empty', 23: 'empty', 24: 'empty'}

In [ ]:
#TODO:
#1. Notri gre load_profil
#2. Ven pride premaknjen load profil 



def shift_load(agent, network, time_frame):
    # search for flexible load
    hours_shifted = 0
    if network.nodes[agent]['schedule'][time_frame] == 'flexible':
        flexibility_treshold = 0.4 # this treshold will move with prikladonst opravila

        # if agent is in first group do that
        if network.nodes[agent]['group'] == 'first' and network.nodes[agent]['flexibility_coeficient'] > flexibility_treshold :
            hours_shifted = 6
            network.nodes[agent]['schedule'][time_frame] = 'shifted'
            network.nodes[agent]['schedule'][time_frame + hours_shifted] = 'flexible_after_shifting'
            network.nodes[agent]['shifted_hours'] =+ hours_shifted

        # if agent is in second group do that
        elif network.nodes[agent]['group'] == 'second' and network.nodes[agent]['flexibility_coeficient'] > flexibility_treshold :
            hours_shifted = 3
            network.nodes[agent]['schedule'][time_frame] = 'shifted'
            network.nodes[agent]['schedule'][time_frame + hours_shifted] = 'flexible_after_shifting'
            network.nodes[agent]['shifted_hours'] =+ hours_shifted

        # if agent is in third group do that
        elif test.nodes[agent]['group'] == 'third' and network.nodes[agent]['flexibility_coeficient'] > flexibility_treshold :
            hours_shifted = 1
            network.nodes[agent]['schedule'][time_frame] = 'shifted'
            network.nodes[agent]['schedule'][time_frame + hours_shifted] = 'flexible_after_shifting'
            network.nodes[agent]['shifted_hours'] =+ hours_shifted

    
    #return network.nodes[agent]['shifted_hours']

In [ ]:
#shift_load(1, test, 18)
print(test.nodes[1])

def restart_schedule(agent, network):
    network.nodes[agent]['schedule'] = test_schedule

#restart_schedule(1,test)
#test.nodes[1]

In [ ]:
test.nodes[2]['susceptibility']

# Obnašanje agentov

In [ ]:
def behaviour(agent, agent_network):
     # creat list of agetns neigbours
    neighbour_list = agent_network.nodes[agent]['neighbours']

    # create empty list and store behaviour of each neighbour
    # in this case that is flexibility_coeficient
    behaviour_list = []
    weight_list = []

    price_signal = 0.9
    shift_signal = 0.9

    # collect neighbours behaviour and weights and add them to the lists
    for neighbour in neighbour_list:
        behaviour_list.append(agent_network.nodes[neighbour]['flexibility_coeficient'])
        weight_list.append(agent_network.nodes[neighbour]['group_weight'])
       
    
    agent_current_weight = agent_network.nodes[agent]['group_weight']# get current agent weight
    agent_current_flexibility = agent_network.nodes[agent]['flexibility_coeficient']# get current agent flexibility 
    print('Agents start flexibility', agent_current_flexibility)

    # adjust weight with the weights of neighbours [Douw Komunikacija]
    #for weight in weight_list:
     #   agent_new_weight = agent_current_weight *(1-agent_network.nodes[agent]['susceptibility'])+weight*agent_network.nodes[agent]['susceptibility']
     #   agent_current_weight = agent_new_weight
     #   print('Začetna utež:',agent_current_weight,'Sosedova utež',weight, '= trenutna utež:', agent_current_weight)

    

    # update agent's flexibility and weight inside network dictionary
    #agent_network.nodes[agent]['flexibility_coeficient'] = agent_current_flexibility
    #agent_network.nodes[agent]['group_weight'] = agent_current_weight
    

    # BEH DIFFUSION & ECO FEEDBACK
    # set variable B, so it repsresents agent's current behaviour
    B = agent_network.nodes[agent]['flexibility_coeficient']

    # set variable goalB, that represents incentivizied behaviour
    goalB = 0.8

    # calculate new behaviour with feedback effect equasion
    #B = B + (goalB - B)* agent_network.nodes[agent]['susceptibility']

    # calculate behaviour diffusion

    B = B + agent_network.nodes[agent]['susceptibility'] * (sum(behaviour_list[n] * weight_list[n] for n in range(len          (behaviour_list)))/ sum(weight_list) - agent_network.nodes[agent]['flexibility_coeficient'])

    # update agent's behaviour inside network dictionary
    agent_network.nodes[agent]['flexibility_coeficient'] = B
    print('Agents new flexibility', B)

    #return agent_network.nodes[agent]['flexibility_coeficient']
    return agent_current_flexibility

In [ ]:
testni_behaviour = behaviour(1, test)
testni_behaviour

## Simulation

In [ ]:
def simulation(network, days, schedule):
    day_frame1 = {}
    day_frame2 = {}
    key_frame = {}
    for day in range(0,days): #counter za 50 iteracij

        #df = pd.DataFrame(columns =('Day', 'Agent group'))
        export_data = {}
        export_data2 = {}
        export_key_data = {}

        print('Dan št:',day+1)
        for agent in network: #v vsaki časovni enoti urnika nato obravnavamo vse agente, gremo preko seznama agenot in izračunamo obnašanje.
            # potem ni važno kako ali je po urah ali 15 min resolucijo
            #print('Ura: ', key)
            for key in schedule:
                if day == 2: # izvedi samo na 3. dan
                    shift_load(agent, network, key)
                    
                #shifted_hours = network.nodes[agent]['shifted_hours']
                agent_group = network.nodes[agent]['group']
                export_key_data[agent_group, agent] = network.nodes[agent]['shifted_hours']
                key_frame[key] = export_key_data
            
            #agent_group = network.nodes[agent]['group']
            agent_flexibility = network.nodes[agent]['flexibility_coeficient']
            export_data[agent_group, agent] = network.nodes[agent]['shifted_hours']
            export_data2[agent_group, agent] = agent_flexibility
                
            agent_flexibility = round(behaviour(agent, network),3)
            
            #restart_schedule(agent,network)

            day_frame1[day+1] = export_data
            day_frame2[day+1] = export_data2
            

    df = pd.DataFrame.from_dict(day_frame1)
    df2 = pd.DataFrame.from_dict(day_frame2)
    df3_keys = pd.DataFrame.from_dict(key_frame)
    return network, df, df2, df3_keys

In [ ]:
results, exported_data, exported_data2, exported_keys = simulation(test, 30, test_schedule)

In [ ]:
#export za premik ur 
exported_data

In [ ]:

exported_data.to_excel('output_obnašanjeagentov1.xlsx')

#df = pd.DataFrame.from_dict(exported_data, orient='index')
#df.set_index(['Agent_group', 'Agent'],)

In [ ]:
#export za flexibility coeficient
exported_data2

In [ ]:
exported_data2.to_excel('output_obnašanjeagentov2.xlsx')

In [ ]:

fig = plt.figure(figsize =(10, 7))
bp = plt.boxplot(exported_data2,showmeans=False, showfliers=False)
plt.xlabel("Čas")
plt.ylabel("Koeficient prožnosti ")
plt.title("Susceptibility = 0.1")
#plot = exported_data2.boxplot(grid=False)


In [ ]:
exported_data2.index.names = ['Group','Id']
#exported_data2.groupby(['Group']).mean()
#type(exported_data2)
#sns.pointplot(data=exported_data2.groupby('Group', as_index=False).mean(), ax=bp)

In [ ]:
exported_keys

In [ ]:
exported_keys.to_excel('output_obnašanjeagentov_keys.xlsx')

# Testi in načrti

Načrt modela: 
1. Ustvari agent data
2. Create agents as nodes in graph (unconnected)
3. Decision logics for shifting electrical loads
4. Calculate behaviour for each agent each time frame
    1. What behaviour consist of? 
    2. 

## Odločitvena logika

In [ ]:
#najdi fleksibilno porabo in jo premakni
def shiftLoad(agent, timeFrame):
    if load == flexible:
        switch loads for fixed
        if group1 == true:
            hours_shifted = 6 
            schedule[timeFrame] = 'shifted_load'
            schedule[timeFrame+hours_shifted] = 'flexible'
        elif group2 == true:
            hours_shifted = 3 
            schedule[timeFrame] = 'shifted_load'
            schedule[timeFrame+hours_shifted] = 'flexible'
    total_hoursh_shifted += hours_shifted
    return(agent, hours_shifted)


## Test poganjanja časa v simmulaciji
Najprej na ure, nato na 15 min resolucijo

In [ ]:

#test zanke za računanje dni

days = 1 
tick_counter = 0 
hour_counter = 0
while tick_counter < days * 96:
    while tick_counter < 1:
        hour_counter = tick_counter % 24 
        print('hour: ', hour_counter)
        tick_counter += 1
        print('15min: ',tick_counter)



##Najbolj smiselsna zanka

for day in range(0,50): #counter za 50 iteracij
    # za vsak dan hočemo skozi urnik -> Sepravi na koliko enot je razdeljen schedule.
    # potem ni važno kako ali je po urah ali 15 min resolucijo, ker se gleda vrstice s
    for row in schedule:
        #v vsaki časovni enoti urnika nato obravnavamo vse agente, gremo preko seznama agenot in izračunamo obnašanje.
        for agent in agents:
            calculateBehaviour()
    

In [ ]:
def simulation(network, days, schedule):
    export_data = {} # prazen slovar za izvoz podatkov
    df = pd.DataFrame()
    for day in range(0,days): #counter za 50 iteracij
        print('Dan št:',day+1)
        for key in schedule:  # za vsak dan hočemo skozi urnik -> Sepravi na koliko enot je razdeljen schedule.
            # potem ni važno kako ali je po urah ali 15 min resolucijo

            print('Ura: ', key)
            for agent in network: #v vsaki časovni enoti urnika nato obravnavamo vse agente, gremo preko seznama agenot in izračunamo obnašanje.
                shifted_hours = network.nodes[agent]['shifted_hours']
                export_data['Agent', agent] = shifted_hours
                export_data['Agent_group', agent] = network.nodes[agent]['group']
                shiftLoad(agent, network, key)
                

    return network, export_data

## Testiranje enačb

In [ ]:
flex = 0.2
susc = 0.1
weight_list = [1,0.5,1]
current_weight = 0.5

for weight in weight_list:
    Gw = current_weight *(1-susc)+(weight*susc)
    current_weight = Gw
    print('Gw od uteži',weight, '=', current_weight)
Gw

#Gw = current_weight *(1-susc)+(weight_list[n] for n in range(len(weight_list)))

In [ ]:
print('Gw', Gw)
flex = Gw * flex
flex

In [ ]:
test_load = {1: 0.1, 2: 0.1, 3: 0.1, 4: 0.1, 5: 0.2, 6: 0.1, 7: 0.1, 8: 0.1, 9: 0.1, 10: 0.1, 11: 0.1, 12: 0.1, 13: 0.1, 14: 0.1, 15: 1.5, 16: 1.5, 17: 0.1, 18: 2.0, 19: 3.0, 20: 2.0, 21: 1.0, 22: 0.2, 23: 0.2, 24: 0.1}

In [ ]:
# pride shift signal
# at 13:00 you can shift 0.4 for 6h 

shift_signal = {hour: 2, load: 0.4}

# shift load preveri samo
def shift_load_test(test_load, shift_signal):

    new_load = 0 
    return new_load

In [ ]:
#       
agent = []
behaviour_coeficient = 0.5
suscpetibility = 0.1
signal_cena = 0.1
signal_dolzina = 0.1
w_cena = 0.5
w_dolzina = 0.9
prikladonst = 0 


In [ ]:
prikladonst = w_cena * (1- signal_cena) + w_dolzina * (1- signal_dolzina)
prikladonst